KAGGLE ile hibrit model şeklinde gidilmektedir. Veri işlemesi lokalde yapılırken, Model eğitimi kaggle üzerinden yapılmaktadır.

Bu dosya kaggle da eğitilen modelin kodlarını ve gelişimini arşivlemek açısından birebir kod ve açıklamalarıdır. (Çıktıları Kendiniz Çalıştırabilirsiniz)

In [ ]:
""" VERİ YÜKLEME """

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
import matplotlib.pyplot as plt

# model için hazırladığımız train.data verisi seti
df = pd.read_csv("/kaggle/input/instacart-features-local/train_data.csv")

print(f"✅ Veri yüklendi! Boyut: {df.shape}")
df.head()

Create_Dataset dosyası ile son hale getirdiğimiz eğitim için veri setini sisteme yükledik ve burada çekiyoruz.

In [ ]:
""" DEĞİŞKENLERİN HAZIRLANMASI """

X = df.drop(['user_id', 'product_id', 'reordered'], axis=1) # değişkenler
y = df['reordered'] # hedef

# %80 Eğitim, %20 test
print("🔪 Veri setleri ayrılıyor...")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Eğitim Seti: {X_train.shape}")
print(f"Test Seti:   {X_val.shape}")

Modelin kullanıcı, ürün ve tahmin etmesi gereken (hedef değişken) sütunlarını görmemesi için kaldırıyoruz diğerlerini alacak. eğer kaldırmaz isek model bunları baz alarak öğrenmek yerine ezber yapar. Yani ahmet şunu almış, muz bu kadar alınıyor gibi ezberden kaçınsın. Eğer ürün popülerse ve kullanıcı daha önce çok aldıysa cevap Evet' yani tekrar alır demesi lazım bu mantıkla yaklaşmalı.


In [ ]:
""" XGBOOST MODELİNİN AYARLARI """

model = xgb.XGBClassifier(
    objective='binary:logistic', # İkili sınıflandırmadır. (binary ikili durumu ifade eder evet-hayır,alacak-almıycak gibi, logistic ise sınıflandırma için kullanırız direkt 0,1 vermesin küsüratlıda versin)
    eval_metric='logloss', # puan sistemi (Çok eminse ve yanlış çıkarsa büyük ceza gibi tahminle orantılı ceza)
    n_estimators=1000,      # Ağaç sayısı
    learning_rate=0.05,     # Öğrenme hızı
    max_depth=6,            # Ağaç derinliği
    tree_method='hist',     # Hızlandırma için (histogram ile veriyi gruplayarak işlem yapsın. hız katar.)
    device='cuda',          # GPU kullanımı (Eğer GPU açtıysan 'cuda', açmadıysan 'cpu' yap)
    early_stopping_rounds=50 # Eğer 50 tur boyunca iyileşme olmazsa dur
)

print("🚀 Model eğitimi başlıyor...")
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)], # her ağaçtan sonra görmediği verilerle test yapra
    verbose=100  # Her 100 adımda(ağaçta) bir durum raporu ver
)